In [1]:
%load_ext autoreload
%autoreload 2
import os 
import pandas as pd
import numpy as np
os.chdir('..')
from src.data.make_dataset import get_preprocessed_datasets
from src.features.build_features import FeatureEngineering

In [2]:
df_bitcoin, df, df_test = get_preprocessed_datasets()

2019-11-13 13:34:15,218 - src.data.make_dataset - INFO - making final data sets from raw data
2019-11-13 13:34:15,454 - src.data.make_dataset - INFO - Shape of df_bitcoin: (2355, 2)
2019-11-13 13:34:15,455 - src.data.make_dataset - INFO - Shape of df: (4757, 91)
2019-11-13 13:34:15,518 - src.data.make_dataset - INFO - Start preprocessing dataframe
2019-11-13 13:34:15,553 - src.data.make_dataset - INFO - Replacing market_data_current_price_usd 
2019-11-13 13:34:15,571 - src.data.make_dataset - INFO - Replacing market_data_ath_usd 
2019-11-13 13:34:15,584 - src.data.make_dataset - INFO - Replacing market_data_circulating_supply 
2019-11-13 13:34:15,597 - src.data.make_dataset - INFO - Preprocessing done!


In [3]:
df.shape

(4757, 91)

In [16]:
fe = FeatureEngineering(df, df_bitcoin, df_test)

/mnt/d/Workspace/data_science_for_business_lmu/src/features/build_features.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  self.df = pd.concat([df, df_test])


In [19]:
fe.construct_features()

2019-11-13 13:57:04,699 - src.features.build_features - INFO - Transform numerical variable for column transaction_count
2019-11-13 13:57:04,700 - src.features.build_features - INFO - Start filling NA values in transaction_count
2019-11-13 13:57:04,709 - src.features.build_features - INFO - Found 2695 NA values in column transaction_count
2019-11-13 13:57:04,710 - src.features.build_features - INFO - Filled NA values
2019-11-13 13:57:04,724 - src.features.build_features - INFO - Transform numerical variable for column holder_count
2019-11-13 13:57:04,725 - src.features.build_features - INFO - Start filling NA values in holder_count
2019-11-13 13:57:04,733 - src.features.build_features - INFO - Found 2699 NA values in column holder_count
2019-11-13 13:57:04,734 - src.features.build_features - INFO - Filled NA values
2019-11-13 13:57:04,747 - src.features.build_features - INFO - Transform numerical variable for column market_data_current_price_usd
2019-11-13 13:57:04,748 - src.features.b

In [20]:
fe.df_features.dtypes

OBS_ID                                     int64
success                                   object
transaction_count                        float64
holder_count                             float64
market_data_current_price_usd            float64
market_data_ath_usd                      float64
market_data_total_supply                 float64
market_data_circulating_supply           float64
public_interest_stats_bing_matches       float64
labels_categories_0                        int64
labels_country_origin                      int64
labels_ico_data_country_origin             int64
labels_ico_data_hardcap_currency           int64
labels_ico_data_softcap_currency           int64
labels_ico_data_total_raised_currency      int64
dtype: object

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [22]:
X_train, y_train, X_test = fe.get_X_y()

X_train = X_train.values
y_train = y_train.values.astype(int)

In [23]:
X_train, X_val, y_train, y_val = train_test_split(
            X_train, y_train, test_size=0.2, random_state=42)

X_val = X_val.astype(int)

In [32]:
X_train

,transaction_count,holder_count,market_data_current_price_usd,market_data_ath_usd,market_data_total_supply,market_data_circulating_supply,public_interest_stats_bing_matches,labels_categories_0,labels_country_origin,labels_ico_data_country_origin,labels_ico_data_hardcap_currency,labels_ico_data_softcap_currency,labels_ico_data_total_raised_currency
0,48262.141038,13576.305982,1.676610e-03,0.034182,6.565882e+07,8.627700e+13,23900.0,23,79,59,23,15,7
1,975773.000000,109840.000000,2.315610e-01,43587.000000,1.000000e+09,6.008030e+15,686000.0,29,111,82,23,15,10
2,22730.000000,13914.000000,7.173600e-04,0.043750,5.000000e+08,4.400000e+08,27500.0,29,102,76,14,8,10
3,125.000000,51.000000,3.360274e-02,43505.000000,5.000000e+06,0.000000e+00,27500.0,26,79,59,23,15,7
4,16417.000000,6250.000000,8.870100e-04,0.030843,1.000000e+07,0.000000e+00,703000.0,26,79,59,23,15,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4752,16776.000000,4829.000000,2.988830e-03,25965.000000,1.050000e+08,0.000000e+00,62100.0,26,23,59,23,15,7
4753,48262.141038,13576.305982,1.745867e-02,0.740470,1.261440e+07,0.000000e+00,27400.0,19,39,59,23,15,7
4754,48262.141038,13576.305982,8.420000e-02,0.012000,1.275000e+08,0.000000e+00,89000.0,26,79,59,23,15,7
4755,168.000000,34.000000,6.000000e-07,0.003540,2.000000e+11,0.000000e+00,82800.0,26,39,59,23,15,7


In [24]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(3805, 13)
(3805,)
(952, 13)
(952,)


In [25]:
y_train = y_train.astype(int)

In [26]:
model = RandomForestClassifier(n_estimators=1000)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [27]:
y_preds = model.predict(X_val)

In [28]:
from sklearn.metrics import matthews_corrcoef


In [29]:
y_val = y_val.astype(int)

In [30]:
matthews_corrcoef(y_val, y_preds)

0.09148758026851055

## Test Set

In [31]:
X_train, y_train, X_test = fe.get_X_y()
X_train.head()

,transaction_count,holder_count,market_data_current_price_usd,market_data_ath_usd,market_data_total_supply,market_data_circulating_supply,public_interest_stats_bing_matches,labels_categories_0,labels_country_origin,labels_ico_data_country_origin,labels_ico_data_hardcap_currency,labels_ico_data_softcap_currency,labels_ico_data_total_raised_currency
0,48262.141038,13576.305982,0.001677,0.034182,6.565882e+07,8.627700e+13,23900.0,23,79,59,23,15,7
1,975773.000000,109840.000000,0.231561,43587.000000,1.000000e+09,6.008030e+15,686000.0,29,111,82,23,15,10
2,22730.000000,13914.000000,0.000717,0.043750,5.000000e+08,4.400000e+08,27500.0,29,102,76,14,8,10
3,125.000000,51.000000,0.033603,43505.000000,5.000000e+06,0.000000e+00,27500.0,26,79,59,23,15,7
4,16417.000000,6250.000000,0.000887,0.030843,1.000000e+07,0.000000e+00,703000.0,26,79,59,23,15,7


In [33]:
a = list(X_test.columns)

In [34]:
a.remove('OBS_ID')

In [35]:
a

['transaction_count',
 'holder_count',
 'market_data_current_price_usd',
 'market_data_ath_usd',
 'market_data_total_supply',
 'market_data_circulating_supply',
 'public_interest_stats_bing_matches',
 'labels_categories_0',
 'labels_country_origin',
 'labels_ico_data_country_origin',
 'labels_ico_data_hardcap_currency',
 'labels_ico_data_softcap_currency',
 'labels_ico_data_total_raised_currency']

In [36]:
X_test_values = X_test[a].values

In [37]:
X_test_values

array([[3.7600000e+02, 2.6500000e+02, 8.3971000e-04, ..., 2.3000000e+01,
        1.5000000e+01, 7.0000000e+00],
       [7.0408000e+04, 2.3115000e+04, 5.0284000e-02, ..., 2.3000000e+01,
        1.5000000e+01, 7.0000000e+00],
       [4.8262141e+04, 1.3576306e+04, 1.3500000e-02, ..., 2.3000000e+01,
        1.5000000e+01, 7.0000000e+00],
       ...,
       [4.8262141e+04, 1.3576306e+04, 3.0392000e-01, ..., 2.3000000e+01,
        1.5000000e+01, 7.0000000e+00],
       [8.7780000e+04, 7.0199000e+04, 7.6088000e-04, ..., 2.3000000e+01,
        1.5000000e+01, 7.0000000e+00],
       [4.8262141e+04, 1.3576306e+04, 2.8610300e-03, ..., 2.3000000e+01,
        1.5000000e+01, 7.0000000e+00]])

In [57]:
preds_test = model.predict(X_test_values)

In [58]:
pd.DataFrame({"OBS_ID": X_test['OBS_ID'],
             "success": preds_test}).to_csv('data/processed/submission04', index=None)

In [59]:
a = pd.read_csv('data/processed/submission04')

In [60]:
b = pd.read_csv('data/processed/submission03')

In [61]:
a.success.value_caounts()

AttributeError: 'Series' object has no attribute 'value_caounts'

In [62]:
b.success.value_counts()

0    982
1     19
Name: success, dtype: int64